In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install textattack
!pip install -U gensim==4.0.0
!pip install python-Levenshtein

Requirement already up-to-date: gensim==4.0.0 in /usr/local/lib/python3.7/dist-packages (4.0.0)


In [ ]:
# import gensim.downloader as api
# corpus = api.load('text8')
# from gensim.models.fasttext import FastText
# model1 = FastText(corpus, sample = 0, sg = 1)

In [ ]:
# import gensim.downloader as api
# corpus = api.load('text8')
# from gensim.models.word2vec import Word2Vec
# model2 = Word2Vec(corpus, sample = 0, sg = 1)

In [ ]:
from gensim.test.utils import get_tmpfile
from gensim.models.fasttext import FastText
#fname = get_tmpfile("/models/fasttext.model") 
model1 = FastText.load("/content/drive/MyDrive/models/fasttext.model") # Change this location

In [ ]:
from gensim.test.utils import get_tmpfile
from gensim.models.word2vec import Word2Vec
#fname1 = get_tmpfile("/models/word2vec.model") 
model2 = Word2Vec.load("/content/drive/MyDrive/models/word2vec.model") # Change this location

In [ ]:
print(model1.wv.most_similar('cat'))
print(model2.wv.most_similar('cat'))

[('bobcat', 0.759185791015625), ('dog', 0.7208123803138733), ('foxglove', 0.7110776901245117), ('pig', 0.7088143229484558), ('goose', 0.7069101333618164), ('cats', 0.702387273311615), ('beak', 0.7008379697799683), ('catfish', 0.6987316608428955), ('foxhound', 0.696578860282898), ('catnip', 0.6960986852645874)]
[('dog', 0.7193913459777832), ('squirrel', 0.696162760257721), ('leopardus', 0.6945451498031616), ('prionailurus', 0.688106894493103), ('albino', 0.6880608201026917), ('bird', 0.6836060285568237), ('eared', 0.6803243160247803), ('rhinoceros', 0.6769237518310547), ('hyena', 0.6756084561347961), ('badger', 0.6752869486808777)]


In [ ]:
# Import the model
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/albert-base-v2-yelp-polarity")
tokenizer = AutoTokenizer("textattack/albert-base-v2-yelp-polarity")
# Change these

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:12<00:00, 38.0MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpy1mcbmou.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


Framing fasttext attack

In [ ]:
from textattack.transformations import WordSwap

class Swapper(WordSwap): # For fasttext
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        for i in range(10):
            if word.lower() in model1.wv.most_similar(word)[i][0].lower():
                continue # Don't return the exact word
            elif word.isupper():
                return [model1.wv.most_similar(word)[i][0].upper()] # Preserving case
            elif word[0].isupper():
                return [model1.wv.most_similar(word)[i][0].capitalize()] # Preserving Capitalization in words
            else:
                return [model1.wv.most_similar(word)[i][0]]
            
        return [model1.wv.most_similar(word)[0][0]]

In [ ]:
def printexample(attack,n):
  print(attack)
  from collections import deque

  import textattack
  import tqdm
  import time
  from IPython.display import display, HTML
  num_examples = n
  num_remaining_attacks = num_examples
  pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

  worklist = deque(range(0, num_examples))
  worklist_tail = worklist[-1]

  attack_log_manager = textattack.loggers.AttackLogManager()

  load_time = time.time()
        
  num_results = 0
  num_failures = 0
  num_successes = 0
  for result in attack.attack_dataset(dataset, indices=worklist):
      result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
      display(HTML(result_html_str))
      attack_log_manager.log_result(result)
            
      if not isinstance(result, textattack.attack_results.SkippedAttackResult):
          pbar.update(1)
      else:
          worklist_tail += 1
          pbar.update(1)
          worklist.append(worklist_tail)

      num_results += 1

      if (type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult):
          num_successes += 1
                
      if type(result) == textattack.attack_results.FailedAttackResult:
          num_failures += 1
      pbar.set_description(
          "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
      )

  pbar.close()

  attack_log_manager.enable_stdout()
  attack_log_manager.log_summary()
  attack_log_manager.flush()
        
  textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

In [ ]:
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.constraints.overlap import MaxWordsPerturbed
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [RepeatModification(),
               StopwordModification(),MaxWordsPerturbed(max_percent=0.3)]

search_method = GreedyWordSwapWIR(wir_method='weighted-saliency')

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("yelp_polarity", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


textattack: Loading datasets dataset yelp_polarity, split test.


Dataset yelp_polarity downloaded and prepared to /root/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/591dc79cb354c6f051b5bf61c77a446205c98156e01669c6218c9bd6aa15f581. Subsequent calls will reuse this data.


In [ ]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_percent):  0.3
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
)


In [ ]:
for i in range(5,26,5):
  print('Examples for n=',i)
  printexample(attack,i)

  0%|          | 0/5 [00:00<?, ?it/s]

Examples for n= 5
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_percent):  0.3
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:  20%|██        | 1/5 [00:04<00:18,  4.68s/it]

[Succeeded / Failed / Total] 2 / 0 / 2:  40%|████      | 2/5 [00:07<00:11,  3.85s/it]

[Succeeded / Failed / Total] 3 / 0 / 3:  60%|██████    | 3/5 [00:08<00:05,  2.76s/it]

...

[Succeeded / Failed / Total] 16 / 7 / 23:  92%|█████████▏| 23/25 [02:23<00:12,  6.25s/it]

[Succeeded / Failed / Total] 17 / 7 / 24:  96%|█████████▌| 24/25 [02:32<00:06,  6.34s/it]

[Succeeded / Failed / Total] 17 / 8 / 25: 100%|██████████| 25/25 [02:34<00:00,  6.17s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 17     |
| Number of failed attacks:     | 8      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 32.0%  |
| Attack success rate:          | 68.0%  |
| Average perturbed word %:     | 8.19%  |
| Average num. words per input: | 133.36 |
| Avg num queries:              | 242.28 |
+-------------------------------+--------+


textattack: Attack time: 154.14698100090027s


In [ ]:
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import WordSwapMaskedLM
from textattack.shared import Attack
from textattack.datasets import HuggingFaceDataset
transformation = WordSwapMaskedLM(
            method="bae", max_candidates=50, min_confidence=0.0
        )
        #
        # Don't modify the same word twice or stopwords.
        #
constraints = [RepeatModification(), StopwordModification()]

        # For the R operations we add an additional check for
        # grammatical correctness of the generated adversarial example by filtering
        # out predicted tokens that do not form the same part of speech (POS) as the
        # original token t_i in the sentence.
constraints.append(PartOfSpeech(allow_verb_noun_swap=True))

        # "To ensure semantic similarity on introducing perturbations in the input
        # text, we filter the set of top-K masked tokens (K is a pre-defined
        # constant) predicted by BERT-MLM using a Universal Sentence Encoder (USE)
        # (Cer et al., 2018)-based sentence similarity scorer."
        #
        # "[We] set a threshold of 0.8 for the cosine similarity between USE-based
        # embeddings of the adversarial and input text."
        #
        # [from email correspondence with the author]
        # "For a fair comparison of the benefits of using a BERT-MLM in our paper,
        # we retained the majority of TextFooler's specifications. Thus we:
        # 1. Use the USE for comparison within a window of size 15 around the word
        # being replaced/inserted.
        # 2. Set the similarity score threshold to 0.1 for inputs shorter than the
        # window size (this translates roughly to almost always accepting the new text).
        # 3. Perform the USE similarity thresholding of 0.8 with respect to the text
        # just before the replacement/insertion and not the original text (For
        # example: at the 3rd R/I operation, we compute the USE score on a window
        # of size 15 of the text obtained after the first 2 R/I operations and not
        # the original text).
        # ...
        # To address point (3) from above, compare the USE with the original text
        # at each iteration instead of the current one (While doing this change
        # for the R-operation is trivial, doing it for the I-operation with the
        # window based USE comparison might be more involved)."
        #
        # Finally, since the BAE code is based on the TextFooler code, we need to
        # adjust the threshold to account for the missing / pi in the cosine
        # similarity comparison. So the final threshold is 1 - (1 - 0.8) / pi
        # = 1 - (0.2 / pi) = 0.936338023.
use_constraint = UniversalSentenceEncoder(
            threshold=0.936338023,
            metric="cosine",
            compare_against_original=True,
            window_size=15,
            skip_text_shorter_than_window=True,
        )
constraints.append(use_constraint)
        #
        # Goal is untargeted classification.
        #
goal_function = UntargetedClassification(model_wrapper)
        #
        # "We estimate the token importance Ii of each token
        # t_i ∈ S = [t1, . . . , tn], by deleting ti from S and computing the
        # decrease in probability of predicting the correct label y, similar
        # to (Jin et al., 2019).
        #
        # • "If there are multiple tokens can cause C to misclassify S when they
        # replace the mask, we choose the token which makes Sadv most similar to
        # the original S based on the USE score."
        # • "If no token causes misclassification, we choose the perturbation that
        # decreases the prediction probability P(C(Sadv)=y) the most."
        #
search_method = GreedyWordSwapWIR(wir_method="delete")

attack=Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("yelp_polarity", None, "test") # Change the dataset

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using /tmp/tfhub_modules to cache modules.
Downloaded https://tfhub.dev/google/universal-sentence-encoder/4, Total size: 987.47MB
Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.
textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Reusing dataset yelp_polarity (/root/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/591dc79cb354c6f051b5bf61c77a446205c98156e01669c6218c9bd6aa15f581)
textattack: Loading datasets dataset yelp_polarity, split test.


In [ ]:
for i in range(5,26,5):
  print('Examples for n=',i)
  printexample(attack,i)

  0%|          | 0/5 [00:00<?, ?it/s]

Examples for n= 5
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bae
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  50
    (min_confidence):  0.0
  )
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.936338023
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:  20%|██        | 1/5 [00:10<00:40, 10.04s/it]

[Succeeded / Failed / Total] 1 / 1 / 2:  40%|████      | 2/5 [00:22<00:34, 11.47s/it]

[Succeeded / Failed / Total] 2 / 1 / 3:  60%|██████    | 3/5 [00:23<00:15,  7.92s/it]

...

[Succeeded / Failed / Total] 10 / 13 / 23:  92%|█████████▏| 23/25 [06:53<00:35, 17.97s/it]

[Succeeded / Failed / Total] 11 / 13 / 24:  96%|█████████▌| 24/25 [07:08<00:17, 17.86s/it]

[Succeeded / Failed / Total] 11 / 14 / 25: 100%|██████████| 25/25 [07:15<00:00, 17.40s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 11     |
| Number of failed attacks:     | 14     |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 56.0%  |
| Attack success rate:          | 44.0%  |
| Average perturbed word %:     | 4.72%  |
| Average num. words per input: | 133.36 |
| Avg num queries:              | 287.24 |
+-------------------------------+--------+


textattack: Attack time: 435.0749018192291s
